# Use options

Options provided for workflow can be created via "options" classes.

There are two categories of options:

* **ExperimentOptions**:
    These options are used to set some parameters for the experiment. For example, the number of experiment shots (`counts`),
    the averaging mode (`averaging_mode`), the acquisition type (`acquisition_type`) and so on. 

* **WorkflowOptions**:
    These options are used to set parameters for the workflow. These include the settings for operating the workflow, such as `logbook` which specifies the protocol for storing a collection of records of workflow execution.
    In addition, options for the constituent tasks of the workflow can also be set by specifying the task name and the options for that task.



In [ ]:
from __future__ import annotations

from laboneq_applications.experiments.options import (
    BaseExperimentOptions,
    TuneupExperimentOptions,
)
from laboneq_applications.workflow import WorkflowOptions, task, workflow

# Create a new experiment options class
The options for an experiment can be created by inheriting from `BaseExperimentOptions` class.

It is recommended to create options classes always with default values. This way, options can be used without specifying them, and the default values will be used.

In [ ]:
class NewExperimentOptions(BaseExperimentOptions):
    operand: int = 1

In [ ]:
opt = NewExperimentOptions()

In [ ]:
opt

If the options classes are defined without default values, an error will be raised when the options instance is created.

In [ ]:
class InvalidExperimentOptions(BaseExperimentOptions):
    operand: int

In [ ]:
opt = InvalidExperimentOptions()

# Standard experiment options classes

The library provides a few standard options classes, such as `TuneupExperimentOptions`, which can be used for typical tune-up experiments or as a starting point for creating new options classes for more advanced experiments.

In [ ]:
opt = TuneupExperimentOptions()
opt

# Create a new workflow options class

Workflow options classes are used to set the options for the workflow and its constituent tasks. These classes must inherit from `WorkflowOptions`, as shown in the cell below. 

The options for the constituent tasks of the workflow can be set by specifying the task name and the options class used for that task.
As in the case of the experiment options, it is recommended to create the workflow options class with default values.

For example, in the cell below, the line `add: NewExperimentOptions = NewExperimentOptions()` sets the type `NewExperimentOptions` and default value `NewExperimentOptions()` for the options of task `add`:

In [ ]:
class NewWorkflowOptions(WorkflowOptions):
    add: NewExperimentOptions = NewExperimentOptions()
    multiply: NewExperimentOptions = NewExperimentOptions()

In [ ]:
workflow_opt = NewWorkflowOptions()

In [ ]:
workflow_opt.add.operand = 2

# Enable the options feature in the workflow

The advantage of using the options feature of a workflow is the automatic passing of options to the tasks and other features like run_until, etc.

At the moment, the options feature is enabled by default in the workflow when the arguments include `options`. In addition, the types for `options` must conform to a certain rule.

Let's illustrate the concepts via an example. 

`myworkflow` contains task `mytask` which takes in an argument and options. 

To use `myworkflow` with options, we must provide the right types for the `options` argument in `myworkflow`.

The supported types are:
- `WorkflowOptionsA | None = None`
- `Union[WorkflowOptionsA, None] = None`
- `Optional[WorkflowOptionsA]`

where `WorkflowOptionsA` is a subclass of `WorkflowOptions`.

From Python 3.10 onward, it is recommended to use `WorkflowOptionsA | None` to conform with the standard practice.

On Python 3.9, `from _future_ import annotations` must be imported to use `WorkflowOptionsA | None`.


In [ ]:
@task
def add(x, options: NewExperimentOptions|None = None):
    return x + options.operand


@task
def multiply(x, options: NewExperimentOptions| None = None):
    return x * options.operand

In [ ]:
@workflow
def myworkflow(options: NewWorkflowOptions | None = None):
    add(x=1)
    multiply(x=2)

# Disallowed types for options

When the type provided for the options includes a subclass of `WorkflowOptions`, we assume that users are attempting to use the options feature. Hence, if the specified type does not follow the form of the above-mentioned types, an error will be raised to inform the user about this, as shown below. 

Note: users are allowed to define and use their own options, passed as any standard Python type (`str`, `dict` etc.). See 'Manual handling of options' below. 

In [ ]:
# an error will be raised


@workflow
def invalid_workflow(options: NewWorkflowOptions | str):
    add(x=1)
    multiply(x=2)

# Run the workflow with options

Create an options object and pass it to the workflow when running it.

In [ ]:
workflow_opt = NewWorkflowOptions()

In [ ]:
workflow_opt.add.operand = 3
workflow_opt.multiply.operand = 3

In [ ]:
res = myworkflow(options=workflow_opt).run()
res.tasks[1].output

When the options input is not provided to the workflow, the default values of `NewWorkflowOptions` will be used

In [ ]:
res = myworkflow().run()
res.tasks[1].output